# Week 1 Assignment

#### Objective

Fetch the iris dataset ([Link](https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data)) (e.g. from the above link) and execute the necessary preprocessing to convert the input features and target labels to a form suitable for inserting into neural networks built with Keras.

Task is to build a multiclass classifier with Keras, that takes the four numerical features of the iris samples as input, and outputs the prediction for its species. The classifier should be a simple neural network model, and consist of 1) no more than one hidden layer, and 2) the output layer of three units with softmax activation.

#### Data preparation

Dataset is fetched from machine learning database, converted to dataframe, attach labels ```'sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'```, shuffle dataset, one hot encode class feature, separate class feature from dataset, get test dataset to try model with it.

In [87]:
import keras.callbacks
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from io import StringIO
import requests
import pandas as pd
import numpy as np

# Data fetch from url
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
response = requests.get(url)

# Data preparation
data = pd.read_csv(StringIO(response.text), header=None, names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'])

# Shuffle dataset
data = data.sample(frac=1).reset_index(drop=True)

# One hot encode
data['class'] = pd.Categorical(data['class']).codes

# Split to x & y sets
X = pd.DataFrame(data.drop(['class'], axis=1))
y = to_categorical(data['class'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Modeling

In [88]:
# Modeling
model = Sequential([
    Input(shape=(4,)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_34 (Dense)            (None, 32)                160       
                                                                 
 dense_35 (Dense)            (None, 3)                 99        
                                                                 
Total params: 259
Trainable params: 259
Non-trainable params: 0
_________________________________________________________________


In [89]:
# Callback
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=4,
    min_delta=0.001,
    restore_best_weights=True
)

# Fitting the model
history = model.fit(X, y, epochs=150, batch_size=16, validation_split=0.2, callbacks=[early_stop])

training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']


# Evaluate model
loss, accuracy = model.evaluate(X, y)
print(f'\nAccuracy: {accuracy}')
print(f'Loss: {loss}')

Epoch 1/150
8/8 [==============================] - 1s 56ms/step - loss: 1.2302 - accuracy: 0.3417 - val_loss: 1.1185 - val_accuracy: 0.3000
Epoch 2/150
8/8 [==============================] - 0s 16ms/step - loss: 0.9813 - accuracy: 0.3583 - val_loss: 0.9296 - val_accuracy: 0.5000
Epoch 3/150
8/8 [==============================] - 0s 13ms/step - loss: 0.8653 - accuracy: 0.6250 - val_loss: 0.8374 - val_accuracy: 0.6333
Epoch 4/150
8/8 [==============================] - 0s 17ms/step - loss: 0.8060 - accuracy: 0.7250 - val_loss: 0.7909 - val_accuracy: 0.6333
Epoch 5/150
8/8 [==============================] - 0s 17ms/step - loss: 0.7774 - accuracy: 0.6667 - val_loss: 0.7568 - val_accuracy: 0.6667
Epoch 6/150
8/8 [==============================] - 0s 16ms/step - loss: 0.7405 - accuracy: 0.6750 - val_loss: 0.7260 - val_accuracy: 0.6333
Epoch 7/150
8/8 [==============================] - 0s 9ms/step - loss: 0.7123 - accuracy: 0.7833 - val_loss: 0.7026 - val_accuracy: 0.6667
Epoch 8/150
8/8 [====

In [90]:
# Make predictions on test data
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

1/1 [==============================] - 0s 50ms/step
